Step 1:Install Beautifulsoup4 lxml parser, 
       Import requests, pandas, numpy, beautiful soup

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes 
!conda install -c conda-forge lxml --yes

import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
print("Libraries Imported!")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries Imported!


Step 2:
1. Read the website source as text using requests.get and store in a variable
2. Import BeautifulSoup package
3. Use BeautifulSoup and parse the html source text with lxml parser 
4. Use BeautifulSoup Prettify method to arrange the tags to have more readability
5. Use soup.find to get the table data within the table tag 
6. print the number of tables, number of rows, number of columns and number of data elements in the webpage

In [17]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml')
soup.prettify()

table = soup.find("table",{"class" : "wikitable sortable"})

# number of tables in a webpage
print(len(table.find_all('tbody')))
# number of rows in the table
print(len(table.find_all('tr')))
# number of columns of the table
print(len(table.find_all('th')))
# total number of data values in the table
print(len(table.find_all('td')))

1
290
3
867


Step 3:
Get the number of rows, number of columns and the column names of the table in the web page

In [18]:
n_columns = 0
n_rows=0
column_names = []

# Find number of rows and columns
for row in table.find_all('tr'):
                
    # Determine the number of rows in the table
    td_tags = row.find_all('td')
    if len(td_tags) > 0:
        n_rows+=1
        if n_columns == 0:
            # Set the number of columns for our table
            n_columns = len(td_tags)
                        
    # Handle column names if we find them
    th_tags = row.find_all('th') 
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text())
print(n_rows)    
print(n_columns)
print(column_names)

289
3
['Postcode', 'Borough', 'Neighbourhood\n']



Step 4: Use 2 for loops to parse through the html tags and get the table data into the pandas dataframe


In [19]:
columns = column_names if len(column_names) > 0 else range(0,n_columns)
df = pd.DataFrame(columns = columns,
                    index= range(0,n_rows))
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        if (column_marker == 2):
         df.iat[row_marker,column_marker] = column.get_text()[:-1]
        else:
         df.iat[row_marker,column_marker] = column.get_text()     
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Step 5:
1. We see that when we scraped the column names from the webpage the carriage return '/n' has also got appended 
2. Let us remove it by renaming the column name and make sure we check it


In [20]:
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

Step 6: Replace all 'Not assigned' to np.nan


In [21]:
df.replace("Not assigned", np.nan, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Step 7: Only process the cells that have an assigned borough. Ignore cells with a Borough that is Not assigned.

In [22]:
df.dropna(subset=["Borough"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Step 8:
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood 
columns will be Queen's Park



In [23]:
df["Neighbourhood"].replace(np.nan,df['Borough'],inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Step 9: Make the columns to str type for aggregating with a comma using join on Neighbourhood column based on Postcode values

In [24]:
df[['Postcode', 'Borough', 'Neighbourhood']] = df[['Postcode', 'Borough', 'Neighbourhood']].astype(str)

Step 10 :More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma

In [25]:
df_agg=df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df_agg

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Step 11: Using Shape function to know the size of the data

In [27]:
df_agg.shape

(103, 3)

In [30]:
df_agg.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
df_agg.shape

(103, 3)

In [32]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Downloading the datafile with the Postal Codes of Canada with there corresponding Latitude and Longitudes

In [42]:
path= 'http://cocl.us/Geospatial_data'
df_coord = pd.read_csv(path)
print('Co-ordinates of Postal Code Datafile downloaded!')
df_coord.head()

Co-ordinates of Postal Code Datafile downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Correcting the Column Names so that the two dataframes can be merged based on the common field 'Postcode'

In [43]:
df_coord.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_coord.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

Merging the two dataframes based on the common field Postcode

In [49]:
df_can = pd.merge(df_agg, df_coord, on="Postcode")
df_can.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Identifying unique Borough's in Canada from the dataframe

In [50]:
df_can['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

Selecting only the neighbourhoods in Toronto

In [51]:
df_toronto = df_can[df_can['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
